In [ ]:
import sys
!{sys.executable} -m pip install pdf2image


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [14]:
import sys
!{sys.executable} -m pip install byaldi qwen-vl-utils transformers #ipywidgets jinja2 einops seaborn

  Using cached Byaldi-0.0.7-py3-none-any.whl.metadata (20 kB)
  Using cached qwen_vl_utils-0.0.14-py3-none-any.whl.metadata (9.0 kB)
  Using cached transformers-4.57.1-py3-none-any.whl.metadata (43 kB)
  Using cached colpali_engine-0.3.12-py3-none-any.whl.metadata (32 kB)
  Using cached ml_dtypes-0.5.3-cp313-cp313-macosx_10_13_universal2.whl.metadata (8.9 kB)
  Using cached mteb-1.6.35-py3-none-any.whl.metadata (23 kB)
  Using cached ninja-1.13.0-py3-none-macosx_10_9_universal2.whl.metadata (5.1 kB)
  Using cached srsly-2.5.1-cp313-cp313-macosx_11_0_arm64.whl.metadata (19 kB)
  Using cached torch-2.9.0-cp313-none-macosx_11_0_arm64.whl.metadata (30 kB)
  Using cached datasets-4.3.0-py3-none-any.whl.metadata (18 kB)
  Using cached jsonlines-4.0.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached numpy-2.3.4-cp313-cp313-macosx_14_0_arm64.whl.metadata (62 kB)
  Using cached requests-2.32.5-py3-none-any.whl.metadata (4.9 kB)
  Using cached scikit_learn-1.7.2-cp313-cp313-macosx_12_0_arm64.wh

In [19]:
import sys
!{sys.executable} -m pip install einops seaborn

  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached matplotlib-3.10.7-cp313-cp313-macosx_11_0_arm64.whl.metadata (11 kB)
  Using cached contourpy-1.3.3-cp313-cp313-macosx_11_0_arm64.whl.metadata (5.5 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.60.1-cp313-cp313-macosx_10_13_universal2.whl.metadata (112 kB)
  Using cached kiwisolver-1.4.9-cp313-cp313-macosx_11_0_arm64.whl.metadata (6.3 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)
Using cached matplotlib-3.10.7-cp313-cp313-macosx_11_0_arm64.whl (8.1 MB)
Using cached contourpy-1.3.3-cp313-cp313-macosx_11_0_arm64.whl (274 kB)
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
Using cached fonttools-4.60.1-cp313-cp313-macosx_10_13_universal2.whl (2.8 MB)
Using cached kiwisolver-1.4.9-cp313-cp313-macosx_11_0_arm64.whl (64 kB)
Using cached pyparsing-3.2.5-py3-none-any.whl (113 kB)

[not

In [ ]:
from pdf2image import convert_from_path
from PIL import Image
import os

def convert_pdf_to_images(pdf_path):
    """Converts a local PDF file into a list of PIL.Image objects."""
    print(f"Converting {pdf_path} to images...")
    images = convert_from_path(pdf_path)
    return images


In [4]:
pdf_file_path = "../Databases for GenAI.pdf"
page_images = convert_pdf_to_images(pdf_file_path)

Converting ../Databases for GenAI.pdf to images...
Successfully converted 20 pages.
Successfully converted 20 pages.


In [20]:
from colpali_engine.models import ColQwen2, ColQwen2Processor
from colpali_engine.models import ColPali, ColPaliProcessor
# from colpali_engine.interpretability import (
#     get_similarity_maps_from_embeddings,
#     plot_all_similarity_maps,
#     plot_similarity_map,
# )

#vison language models
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
from qwen_vl_utils import process_vision_info

import torch
from torch.utils.data import DataLoader
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

import os
from tqdm import tqdm
import base64
from io import BytesIO
#import matplotlib.pyplot as plt
import pprint


In [22]:
model_name = "vidore/colpali-v1.2"
processor = ColPaliProcessor.from_pretrained(model_name)

model = ColPali.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    device_map=device,
).eval()

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 10.29it/s]


In [23]:
def create_document_embeddings(page_images, model, processor, batch_size=2):
    
    all_embeddings_with_metadata = []

    dataloader = DataLoader(
            dataset=page_images,
            batch_size=batch_size,
            shuffle=False,
            collate_fn=lambda x: processor.process_images(x),
        )

    page_counter = 0
    for batch in tqdm(dataloader):
        with torch.no_grad():
            batch = {k: v.to(model.device) for k, v in batch.items()}
            batch_embeddings = model(**batch)
            batch_embeddings = list(torch.unbind(batch_embeddings.to("cpu")))

            for embedding in batch_embeddings:
                all_embeddings_with_metadata.append({
                    "embedding": embedding,
                    "page_id": page_counter,
                })
                page_counter += 1

    return all_embeddings_with_metadata


ds = create_document_embeddings(page_images, model, processor, batch_size=2)
print(f"Generated embeddings for {len(ds)} pages.")

100%|██████████| 10/10 [11:41<00:00, 70.17s/it]

Generated embeddings for 20 pages.


In [24]:
def get_results(query, processor, model, ds, all_images, top_k=5):
    
    # Process the query and move to model's device
    batch_queries = processor.process_queries([query]).to(model.device)

    # Forward pass to get query embeddings
    with torch.no_grad():
        query_embeddings = model(**batch_queries)

    # Extract embeddings from ds for scoring
    document_embeddings = torch.stack([entry["embedding"] for entry in ds])
    
    # Compute similarity scores
    scores = processor.score_multi_vector(query_embeddings, document_embeddings)
    score_values = scores[0].tolist()  # Extract similarity scores as a list

    # Get top-k indices of the most relevant embeddings
    top_indices = scores[0].topk(top_k).indices.tolist()

    # Retrieve corresponding images and metadata
    retrieved_results = []
    for idx in top_indices:
        entry = ds[idx]
        page_id = entry["page_id"]
        image = all_images[page_id]

        # Add score to each result
        retrieved_results.append({
            "page_id": page_id,
            "image": image,
            "score": score_values[idx],  # Add similarity score
        })

        return retrieved_results


In [ ]:
vl_model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype=torch.bfloat16).to(device).eval()
min_pixels = 224 * 224
max_pixels = 1024 * 1024
vl_model_processor = Qwen2VLProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct", min_pixels=min_pixels, max_pixels=max_pixels
)

In [ ]:
def generate_answers(vl_model,vl_model_processor,query,max_new_tokens=500):
    
    retrieved_results = get_results(
        query = query,
        processor=processor,
        model=model,
        ds=ds,
        all_images=page_images,
        top_k=2,
    )
    grouped_images = [result["image"] for result in retrieved_results]
    chat_template = [
        {
            "role": "user",
            "content": [{"type": "image", "image": image} for image in grouped_images]
            + [{"type": "text", "text": query}],
        }
    ]
    # process the template
    text = vl_model_processor.apply_chat_template(chat_template, tokenize=False, 
                                              add_generation_prompt=True)

    image_inputs, _ = process_vision_info(chat_template)
    inputs = vl_model_processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    ).to(device)

    # generate model output
    generated_ids = vl_model.generate(**inputs, max_new_tokens=max_new_tokens)
    generated_ids_trimmed = [out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)]
    output_text = vl_model_processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return output_text[0]

In [ ]:
query = "Who is the Head of Cloud Platorms & AI Director?"
print(generate_answers(vl_model,vl_model_processor,query,max_new_tokens=500))


[{'page_id': 0,
  'image': <PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=2000x1125>,
  'score': 17.75}]